Collection是一个固定列、可变行的二维表，每列表示一个field，每行表示一个entity。我们可以通过schema来定义数据结构。

# 创建Schema

下面创建**my_id**, **my_vector**, and **my_varchar**三个字段的Schema

In [1]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

schema.add_field(field_name="my_id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="my_vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="my_varchar", datatype=DataType.VARCHAR, max_length=512)

{'auto_id': False, 'description': '', 'fields': [{'name': 'my_id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'my_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 5}}, {'name': 'my_varchar', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}], 'enable_dynamic_field': True}

# 创建索引（Optional）

AUTOINDEX可以对所有vector字段设置索引

In [2]:
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="my_id",
    index_type="STL_SORT"
)

index_params.add_index(
    field_name="my_vector", 
    index_type="AUTOINDEX",
    metric_type="COSINE"
)

# 创建Collection
如果创建了索引，milvus会自动导入索引

In [3]:
# Create a collection with the index loaded simultaneously
client.create_collection(
    collection_name="customized_setup_1",
    schema=schema,
    index_params=index_params
)

res = client.get_load_state(
    collection_name="customized_setup_1"
)

print(res)

{'state': <LoadState: Loaded>}


下面展示创建Collection的时候不加载索引

In [4]:
# Create a collection and index it separately
client.create_collection(
    collection_name="customized_setup_2",
    schema=schema,
)

res = client.get_load_state(
    collection_name="customized_setup_2"
)

print(res)

{'state': <LoadState: NotLoad>}


# 设置Collection的属性
## 设置Shard数量

In [6]:
client.create_collection(
    collection_name="customized_setup_3",
    schema=schema,
    # highlight-next-line
    num_shards=1
)

## 启用mmap
milvus默认使用mmap，这样milvus可以按照field数据加载到内存中，而不是加载全部数据。

In [7]:
client.create_collection(
    collection_name="customized_setup_4",
    schema=schema,
    # highlight-next-line
    enable_mmap=False
)

## 设置Collection TTL
一旦Time-To-Live (TTL) 超时，milvus就是删除Collection中的entities和Collection，默认是一天（86400s）

In [8]:
client.create_collection(
    collection_name="customized_setup_5",
    schema=schema,
    # highlight-start
    properties={
        "collection.ttl.seconds": 86400
    }
    # highlight-end
)

## 设置Consistency Level

In [9]:
client.create_collection(
    collection_name="customized_setup_6",
    schema=schema,
    # highlight-next
    consistency_level="Bounded",
)